# Refine DRKG
This notebook shows how to refine DRKG

You need to download the original DRKG source file into your local storage, e.g., ./data/drkg.tsv

## Install DGL-KE
Before training the model, we need to install dgl and dgl-ke packages as well as other dependencies. 

In [4]:
!sudo pip3 install torch
!sudo pip3 install dgl 
!sudo pip3 install dglke

     |████████████████████████████████| 752.0MB 79.9MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 3.0MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 26.1MB 23.0MB/s eta 0:00:01
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 61kB 2.5MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.


## Prepare train/valid/test set
Before training, we need to split the original drkg into train/valid/test set with a 9:0.5:0.5 manner.

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../utils')
from utils import download_and_extract
download_and_extract()
drkg_file = '../data/drkg/drkg.tsv'

df = pd.read_csv(drkg_file, sep="\t")
triples = df.values.tolist()

We get 5,869,293 triples, now we will split them into three files

In [3]:
num_triples = len(triples)
num_triples
# Please make sure the output directory exist.
import os
#os.mkdir('train_Hetionet')

Apply mapping

In [9]:
def mapping(st):
    slptst=st.split(':')
    if len(slptst)<3:
        return st
    return slptst[-2]+slptst[-1]
for idx in range(num_triples):
        triples[idx][1]=mapping(triples[idx][1])
        #print(triples[idx][1])

In [10]:
# Please make sure the output directory exist.
os.mkdir('train_crude_drkg')
dataset_name='crude_drkg'
with open("train_crude_drkg/drkg_train.tsv", 'w+') as f:
    for idx in range(num_triples):
            f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

## Training TransE_l2 model
We can training the TransE_l2 model by simplying using DGL-KE command line. For more information about using DGL-KE please refer to https://github.com/awslabs/dgl-ke.

Here we train the model using 8 GPUs on an AWS p3.16xlarge instance.

In [11]:
!DGLBACKEND=pytorch dglke_train --dataset crude_drkg --data_path ./train_crude_drkg --data_files drkg_train.tsv --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 4096 \
--neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07  --num_thread 1 --gpu 0 1 2 3 4 5 6 7 --num_proc 8 --neg_sample_size_eval 10000 --async_update

Using backend: pytorch
Logs are being recorded at: ckpts/TransE_l2_crude_drkg_0/train.log
Reading train triples....
Finished. Read 5874260 train triples.
|Train|: 5874260
random partition 5874260 edges into 8 parts
part 0 has 734283 edges
part 1 has 734283 edges
part 2 has 734283 edges
part 3 has 734283 edges
part 4 has 734283 edges
part 5 has 734283 edges
part 6 has 734283 edges
part 7 has 734279 edges
/usr/local/lib/python3.6/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
Total initialize time 23.219 seconds
[proc 6][Train](1000/100000) average pos_loss: 0.7106251298597308
[proc 5][Train](1000/100000) average pos_loss: 0.6795004418045283
[proc 3][Train](1000/100000) average pos_loss: 0.4529614802002907
[proc 6][Train](1000/100000) average neg_loss: 0.689767808392644
[proc 1][Train](1000/100000) average pos_loss: 0.5866258220672608
[proc 3][Train](1000/100000) average neg_l

[proc 2][Train](4000/100000) average pos_loss: 0.3255469411015511
[proc 1][Train](4000/100000) average pos_loss: 0.3249851799607277
[proc 0][Train](4000/100000) average pos_loss: 0.3253792814314365
[proc 6][Train](4000/100000) average pos_loss: 0.32273857048153876
[proc 2][Train](4000/100000) average neg_loss: 0.45861767828464506
[proc 4][Train](4000/100000) average pos_loss: 0.32540505108237266
[proc 3][Train](4000/100000) average pos_loss: 0.32570590162277224
[proc 1][Train](4000/100000) average neg_loss: 0.4598769754171371
[proc 5][Train](4000/100000) average pos_loss: 0.3227359306514263
[proc 2][Train](4000/100000) average loss: 0.3920823102295399
[proc 6][Train](4000/100000) average neg_loss: 0.45988298121094706
[proc 4][Train](4000/100000) average neg_loss: 0.4596274452507496
[proc 0][Train](4000/100000) average neg_loss: 0.4584119304418564
[proc 2][Train](4000/100000) average regularization: 0.06515229100733995
[proc 7][Train](4000/100000) average pos_loss: 0.32516405135393145
[

[proc 1][Train] 1000 steps take 66.983 seconds
[proc 2][Train](6000/100000) average regularization: 0.06507474226504564
[proc 1]sample: 10.201, forward: 36.692, backward: 4.560, update: 11.796
[proc 2][Train] 1000 steps take 66.983 seconds
[proc 2]sample: 9.897, forward: 36.267, backward: 3.920, update: 11.405
[proc 0][Train](7000/100000) average pos_loss: 0.3152560563087463
[proc 6][Train](7000/100000) average pos_loss: 0.31353006640076636
[proc 0][Train](7000/100000) average neg_loss: 0.4490700279176235
[proc 5][Train](7000/100000) average pos_loss: 0.3133018798828125
[proc 4][Train](7000/100000) average pos_loss: 0.31516047203540803
[proc 1][Train](7000/100000) average pos_loss: 0.3152700252830982
[proc 0][Train](7000/100000) average loss: 0.38216304215788843
[proc 5][Train](7000/100000) average neg_loss: 0.44957192584872246
[proc 7][Train](7000/100000) average pos_loss: 0.3149319726526737
[proc 2][Train](7000/100000) average pos_loss: 0.3153228111863136
[proc 6][Train](7000/100000)

[proc 6][Train](10000/100000) average pos_loss: 0.30994282534718515
[proc 5][Train](10000/100000) average pos_loss: 0.3097453978061676
[proc 6][Train](10000/100000) average neg_loss: 0.4454827257692814
[proc 5][Train](10000/100000) average neg_loss: 0.44475050619244577
[proc 6][Train](10000/100000) average loss: 0.377712776273489
[proc 0][Train](10000/100000) average pos_loss: 0.31120089468359946
[proc 6][Train](10000/100000) average regularization: 0.06470196558535099
[proc 5][Train](10000/100000) average loss: 0.3772479518651962
[proc 0][Train](10000/100000) average neg_loss: 0.4443385103940964
[proc 6][Train] 1000 steps take 68.279 seconds
[proc 6]sample: 10.336, forward: 34.167, backward: 3.503, update: 10.592
[proc 3][Train](10000/100000) average pos_loss: 0.3118169237077236
[proc 5][Train](10000/100000) average regularization: 0.06470068233460188
[proc 0][Train](10000/100000) average loss: 0.3777697024643421
[proc 7][Train](10000/100000) average pos_loss: 0.31109086334705355
[pro

[proc 0][Train](13000/100000) average pos_loss: 0.30922982493042944
[proc 2][Train](13000/100000) average pos_loss: 0.30931695276498794
[proc 0][Train](13000/100000) average neg_loss: 0.4422270845770836
[proc 6][Train](13000/100000) average pos_loss: 0.3081205866038799
[proc 2][Train](13000/100000) average neg_loss: 0.44282121086120607
[proc 4][Train](13000/100000) average pos_loss: 0.30917099976539614
[proc 3][Train](13000/100000) average pos_loss: 0.30987056016922
[proc 0][Train](13000/100000) average loss: 0.3757284549176693
[proc 7][Train](13000/100000) average pos_loss: 0.30905343049764633
[proc 2][Train](13000/100000) average loss: 0.3760690814256668
[proc 6][Train](13000/100000) average neg_loss: 0.4434024371504784
[proc 4][Train](13000/100000) average neg_loss: 0.4421216436624527
[proc 1][Train](13000/100000) average pos_loss: 0.3093237216770649
[proc 0][Train](13000/100000) average regularization: 0.06444337502866983
[proc 3][Train](13000/100000) average neg_loss: 0.4433751338

...

[proc 0][Train](88000/100000) average pos_loss: 0.3026479388475418
[proc 7][Train](88000/100000) average pos_loss: 0.3027630689442158
[proc 4][Train](88000/100000) average pos_loss: 0.302541550219059
[proc 0][Train](88000/100000) average neg_loss: 0.43538455027341844
[proc 2][Train](88000/100000) average pos_loss: 0.30280160063505174
[proc 3][Train](88000/100000) average pos_loss: 0.30314995035529135
[proc 1][Train](88000/100000) average pos_loss: 0.3030517347157001
[proc 5][Train](88000/100000) average pos_loss: 0.30228583800792697
[proc 7][Train](88000/100000) average neg_loss: 0.4350010848045349
[proc 4][Train](88000/100000) average neg_loss: 0.4353006283938885
[proc 0][Train](88000/100000) average loss: 0.3690162441730499
[proc 6][Train](88000/100000) average pos_loss: 0.3026252463757992
[proc 2][Train](88000/100000) average neg_loss: 0.43469162434339526
[proc 3][Train](88000/100000) average neg_loss: 0.4356134605407715
[proc 1][Train](88000/100000) average neg_loss: 0.435547310113

[proc 1][Train](91000/100000) average pos_loss: 0.3030663085579872
[proc 7][Train](91000/100000) average pos_loss: 0.3027256300151348
[proc 2][Train](91000/100000) average pos_loss: 0.30270424038171767
[proc 1][Train](91000/100000) average neg_loss: 0.4355233702361584
[proc 5][Train](91000/100000) average pos_loss: 0.3022642059922218
[proc 4][Train](91000/100000) average pos_loss: 0.3025545211136341
[proc 6][Train](91000/100000) average pos_loss: 0.3026607660651207
[proc 0][Train](91000/100000) average pos_loss: 0.3026881241798401
[proc 7][Train](91000/100000) average neg_loss: 0.43515221345424654
[proc 1][Train](91000/100000) average loss: 0.36929483932256696
[proc 2][Train](91000/100000) average neg_loss: 0.4355781590640545
[proc 3][Train](91000/100000) average pos_loss: 0.3031706026792526
[proc 5][Train](91000/100000) average neg_loss: 0.4351033265888691
[proc 4][Train](91000/100000) average neg_loss: 0.43478608602285385
[proc 6][Train](91000/100000) average neg_loss: 0.435290246397

[proc 0][Train](94000/100000) average pos_loss: 0.30259725227952006
[proc 1][Train](94000/100000) average pos_loss: 0.3030568850934505
[proc 2][Train](94000/100000) average pos_loss: 0.30271457690000536
[proc 7][Train](94000/100000) average pos_loss: 0.3028063441812992
[proc 0][Train](94000/100000) average neg_loss: 0.4353335957527161
[proc 1][Train](94000/100000) average neg_loss: 0.4353734711110592
[proc 5][Train](94000/100000) average pos_loss: 0.3022958951294422
[proc 2][Train](94000/100000) average neg_loss: 0.4353138794004917
[proc 4][Train](94000/100000) average pos_loss: 0.30262940537929534
[proc 6][Train](94000/100000) average pos_loss: 0.30269383785128595
[proc 3][Train](94000/100000) average pos_loss: 0.30314323595166204
[proc 0][Train](94000/100000) average loss: 0.36896542409062383
[proc 7][Train](94000/100000) average neg_loss: 0.43495004710555074
[proc 1][Train](94000/100000) average loss: 0.36921517834067347
[proc 5][Train](94000/100000) average neg_loss: 0.435115119844

[proc 3][Train](97000/100000) average pos_loss: 0.303106419891119
[proc 7][Train](97000/100000) average pos_loss: 0.3026762965917587
[proc 4][Train](97000/100000) average pos_loss: 0.30254715514183045
[proc 3][Train](97000/100000) average neg_loss: 0.4346842162311077
[proc 6][Train](97000/100000) average pos_loss: 0.3025589895546436
[proc 7][Train](97000/100000) average neg_loss: 0.43596934270858767
[proc 5][Train](97000/100000) average pos_loss: 0.30223474037647247
[proc 4][Train](97000/100000) average neg_loss: 0.43520046335458756
[proc 2][Train](97000/100000) average pos_loss: 0.302788877338171
[proc 0][Train](97000/100000) average pos_loss: 0.30263601154088976
[proc 1][Train](97000/100000) average pos_loss: 0.30305429676175116
[proc 3][Train](97000/100000) average loss: 0.36889531815052035
[proc 6][Train](97000/100000) average neg_loss: 0.43630131164193153
[proc 4][Train](97000/100000) average loss: 0.36887380909919737
[proc 7][Train](97000/100000) average loss: 0.3693228196203709


[proc 1][Train](100000/100000) average pos_loss: 0.302834195703268
[proc 5][Train](100000/100000) average pos_loss: 0.3021631615459919
[proc 7][Train](100000/100000) average pos_loss: 0.3025906626284122
[proc 1][Train](100000/100000) average neg_loss: 0.43575628834962843
[proc 0][Train](100000/100000) average pos_loss: 0.30245386612415315
[proc 2][Train](100000/100000) average pos_loss: 0.302754376411438
[proc 4][Train](100000/100000) average pos_loss: 0.30247482207417486
[proc 7][Train](100000/100000) average neg_loss: 0.4350108154416084
[proc 5][Train](100000/100000) average neg_loss: 0.4346328747570515
[proc 1][Train](100000/100000) average loss: 0.36929524236917494
[proc 6][Train](100000/100000) average pos_loss: 0.3024376234412193
[proc 3][Train](100000/100000) average pos_loss: 0.30302954104542734
[proc 0][Train](100000/100000) average neg_loss: 0.436008148252964
[proc 2][Train](100000/100000) average neg_loss: 0.4351882646381855
[proc 4][Train](100000/100000) average neg_loss: 0

## Get Entity and Relation Embeddings
The resulting model, i.e., the entity and relation embeddings can be found under ./ckpts. (Please refer to the first line of the training log for the specific location.)

The overall process will generate 4 important files:

  - Entity embedding: ./ckpts/<model\_name>_<dataset\_name>_<run_\id>/xxx\_entity.npy
  - Relation embedding: ./ckpts/<model\_name>_<dataset\_name>_<run\_id>/xxx\_relation.npy
  - The entity id mapping, formated in <entity\_name> <entity\_id> pair: <data\_path>/entities.tsv
  - The relation id mapping, formated in <relation\_name> <relation\_id> pair: <data\_path>/relations.tsv

In [ ]:
!ls ./ckpts/TransE_l2_DRKG_0/
!ls ./train/

## A Glance of the Entity and Relation Embeddings

In [ ]:
node_emb = np.load('./ckpts/TransE_l2_DRKG_1/DRKG_TransE_l2_entity.npy')
relation_emb = np.load('./ckpts/TransE_l2_DRKG_1/DRKG_TransE_l2_relation.npy')

print(node_emb.shape)
print(relation_emb.shape)